In [ ]:
# bugfixes

In [2]:
import os
import json

# Load the config file and set it as JSON string in the environment
with open('../lithops_config.json', 'r') as f:
    config_data = json.load(f)

os.environ['LITHOPS_CONFIG'] = json.dumps(config_data)

from lithops import FunctionExecutor

def check_python(_):
    import sys
    return sys.version

with FunctionExecutor() as fexec:
    future = fexec.call_async(check_python, None)
    result = fexec.get_result([future])
    print(result)

2025-08-26 18:44:10,984 [INFO] config.py:139 -- Lithops v3.6.1 - Python3.12
2025-08-26 18:44:10,984 [INFO] localhost.py:39 -- Localhost storage client created
2025-08-26 18:44:10,985 [INFO] localhost.py:78 -- Localhost compute v2 client created
2025-08-26 18:44:11,205 [INFO] invokers.py:119 -- ExecutorID c2f8b6-2 | JobID A000 - Selected Runtime: /home/jovyan/rasi-icechunk/.venv/bin/python 
2025-08-26 18:44:11,212 [INFO] invokers.py:186 -- ExecutorID c2f8b6-2 | JobID A000 - Starting function invocation: check_python() - Total: 1 activations
2025-08-26 18:44:11,214 [INFO] invokers.py:225 -- ExecutorID c2f8b6-2 | JobID A000 - View execution logs at /tmp/lithops-root/logs/c2f8b6-2-A000.log
2025-08-26 18:44:11,214 [INFO] executors.py:494 -- ExecutorID c2f8b6-2 - Getting results from 1 function activations
2025-08-26 18:44:11,215 [INFO] wait.py:101 -- ExecutorID c2f8b6-2 - Waiting for 1 function activations to complete
/home/jovyan/nldas-icechunk/.venv/lib/python3.12/site-packages/tqdm/auto.

3.12.10 (main, Apr  9 2025, 04:03:51) [Clang 20.1.0 ]


In [5]:
import fsspec
from virtualizarr import open_virtual_mfdataset, open_virtual_dataset
from virtualizarr.parsers import HDFParser
from virtualizarr.registry import ObjectStoreRegistry
import xarray as xr
import obstore

data_dir = "s3://nasa-waterinsight/NLDAS3/forcing/daily/"

# Use fsspec to list files in the S3 bucket
fs = fsspec.filesystem("s3", anon=True)
files = fs.glob(data_dir + "**/*.nc")
print(f"{len(files)} found")
print(f"FOR TESTING: Cutting the number of files")
files = files[0:3]

## Produce a virtual dataset from the list of files
bucket = "s3://nasa-waterinsight"
store = obstore.store.from_url(bucket, region="us-west-2", skip_signature=True)

registry = ObjectStoreRegistry({bucket: store})

parser = HDFParser()

urls = ["s3://" + file for file in files]
vdatasets = [open_virtual_dataset(url, parser=parser, registry=registry, loadable_variables=['time']) for url in urls]

8399 found
FOR TESTING: Cutting the number of files


/home/jovyan/nldas-icechunk/.venv/lib/python3.12/site-packages/numcodecs/zarr3.py:145: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)
/home/jovyan/nldas-icechunk/.venv/lib/python3.12/site-packages/numcodecs/zarr3.py:145: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)
/home/jovyan/nldas-icechunk/.venv/lib/python3.12/site-packages/numcodecs/zarr3.py:145: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)


In [29]:
import fsspec
import xarray as xr

data_dir = "s3://nasa-waterinsight/NLDAS3/forcing/daily/"

# Use fsspec to list files in the S3 bucket
fs = fsspec.filesystem("s3", anon=True)
files = fs.glob(data_dir + "**/*.nc")
files = files[0:3]

datasets = []
for url in urls:
    with fsspec.open(url) as f:
        datasets.append(xr.open_dataset(f,chunks={}))
datasets

8399 found
FOR TESTING: Cutting the number of files


[<xarray.Dataset> Size: 3GB
 Dimensions:   (lon: 11700, lat: 6500, time: 1)
 Coordinates:
   * lon       (lon) float32 47kB -169.0 -169.0 -169.0 ... -52.03 -52.01 -52.0
   * lat       (lat) float32 26kB 7.005 7.015 7.025 7.035 ... 71.97 71.98 71.99
   * time      (time) datetime64[ns] 8B 2001-01-02
 Data variables:
     Tair      (time, lat, lon) float32 304MB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
     Tair_min  (time, lat, lon) float32 304MB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
     Tair_max  (time, lat, lon) float32 304MB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
     Qair      (time, lat, lon) float32 304MB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
     PSurf     (time, lat, lon) float32 304MB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
     LWdown    (time, lat, lon) float32 304MB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
     SWdown    (time, lat, lon) float32 304MB dask.array<chunksize=(1, 500, 900), meta=np

In [30]:
def combine_attrs(dicts, context):
    print("++++++++++++++++++++++++++++++++++++++++++")
    # print(f"{context=}")
    # print(dicts)
    combined_attrs = {}
    
    # Get keys from first dict as reference
    all_keys = set(dicts[0].keys())
    
    # Check that every key exists in all dicts
    for i, d in enumerate(dicts[1:], 1):
        if set(d.keys()) != all_keys:
            missing = all_keys - set(d.keys())
            extra = set(d.keys()) - all_keys
            raise KeyError(f"Dict {i} key mismatch. Missing: {missing}, Extra: {extra}")
    
    for key in all_keys:
        values = [d[key] for d in dicts]
        unique_values = set(values)
        
        if len(unique_values) == 1:
            # All values are the same
            combined_attrs[key] = values[0]
        else:
            if key == 'vmin':
                combined_attrs[key] = min(values)
            elif key == 'vmax':
                combined_attrs[key] = max(values)
            elif key == 'begin_date':
                combined_attrs[key] = min(values)
            elif key == 'end_date':
                combined_attrs[key] = max(values)
            elif key == 'history':
                combined_attrs[key] = '||'.join(values)
            else:
                print(f"HANDLE {key=}!!!")

            if key in combined_attrs.keys():
                print(f"Combined {key} from {values} to {combined_attrs[key]}")
                
    return combined_attrs

xr.concat(datasets, 'time', combine_attrs=combine_attrs)

++++++++++++++++++++++++++++++++++++++++++
Combined history from ['created on date: 2025-06-07T15:04:40', 'created on date: 2025-06-07T15:10:57', 'created on date: 2025-06-07T15:17:23'] to created on date: 2025-06-07T15:04:40||created on date: 2025-06-07T15:10:57||created on date: 2025-06-07T15:17:23
++++++++++++++++++++++++++++++++++++++++++
Combined end_date from ['20010101', '20010102', '20010103'] to 20010103
Combined begin_date from ['20010101', '20010102', '20010103'] to 20010101
++++++++++++++++++++++++++++++++++++++++++
Combined vmin from [np.float32(228.31163), np.float32(227.43488), np.float32(227.1345)] to 227.13450622558594
Combined vmax from [np.float32(304.30316), np.float32(304.46704), np.float32(303.49533)] to 304.467041015625
++++++++++++++++++++++++++++++++++++++++++
Combined vmin from [np.float32(227.16179), np.float32(225.08893), np.float32(223.27415)] to 223.27415466308594
Combined vmax from [np.float32(300.45273), np.float32(300.49905), np.float32(300.90958)] to 3

<xarray.Dataset> Size: 9GB
Dimensions:   (time: 3, lat: 6500, lon: 11700)
Coordinates:
  * lon       (lon) float32 47kB -169.0 -169.0 -169.0 ... -52.03 -52.01 -52.0
  * lat       (lat) float32 26kB 7.005 7.015 7.025 7.035 ... 71.97 71.98 71.99
  * time      (time) datetime64[ns] 24B 2001-01-02 2001-01-03 2001-01-04
Data variables:
    Tair      (time, lat, lon) float32 913MB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
    Tair_min  (time, lat, lon) float32 913MB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
    Tair_max  (time, lat, lon) float32 913MB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
    Qair      (time, lat, lon) float32 913MB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
    PSurf     (time, lat, lon) float32 913MB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
    LWdown    (time, lat, lon) float32 913MB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
    SWdown    (time, lat, lon) float32 913MB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
    Wind_N    (time, lat, lon) float32 913MB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
    Wind_E    (time, lat, lon) float32 913MB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
    Rainf     (time, lat, lon) float32 913MB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
Attributes: (12/17)
    institution:            NASA GSFC
    reference:              Maina-et-al_to-be-submitted; doi:TBD
    SOUTH_WEST_CORNER_LAT:  7.005
    DX:                     0.01
    shortname:              NLDAS_FOR0010_D_3.0
    time_definition:        daily
    ...                     ...
    DY:                     0.01
    websites:               https://ldas.gsfc.nasa.gov/nldas/v3/ ; https://li...
    title:                  NLDAS Forcing Data L4 Daily 0.01 x 0.01 degree V3...
    history:                created on date: 2025-06-07T15:04:40||created on ...
    missing_value:          -9999.0
    doi:                    TBD

In [6]:
# Check each files metadata and timestamp
display(vdatasets[0])
display(vdatasets[-1])

<xarray.Dataset> Size: 3GB
Dimensions:   (time: 1, lat: 6500, lon: 11700)
Coordinates:
  * time      (time) datetime64[ns] 8B 2001-01-02
    lon       (lon) float32 47kB ManifestArray<shape=(11700,), dtype=float32,...
    lat       (lat) float32 26kB ManifestArray<shape=(6500,), dtype=float32, ...
Data variables:
    Tair      (time, lat, lon) float32 304MB ManifestArray<shape=(1, 6500, 11...
    Tair_min  (time, lat, lon) float32 304MB ManifestArray<shape=(1, 6500, 11...
    Tair_max  (time, lat, lon) float32 304MB ManifestArray<shape=(1, 6500, 11...
    Qair      (time, lat, lon) float32 304MB ManifestArray<shape=(1, 6500, 11...
    PSurf     (time, lat, lon) float32 304MB ManifestArray<shape=(1, 6500, 11...
    LWdown    (time, lat, lon) float32 304MB ManifestArray<shape=(1, 6500, 11...
    SWdown    (time, lat, lon) float32 304MB ManifestArray<shape=(1, 6500, 11...
    Wind_N    (time, lat, lon) float32 304MB ManifestArray<shape=(1, 6500, 11...
    Wind_E    (time, lat, lon) float32 304MB ManifestArray<shape=(1, 6500, 11...
    Rainf     (time, lat, lon) float32 304MB ManifestArray<shape=(1, 6500, 11...
Attributes: (12/17)
    missing_value:          -9999.0
    time_definition:        daily
    shortname:              NLDAS_FOR0010_D_3.0
    title:                  NLDAS Forcing Data L4 Daily 0.01 x 0.01 degree V3...
    version:                3.0 beta
    institution:            NASA GSFC
    ...                     ...
    websites:               https://ldas.gsfc.nasa.gov/nldas/v3/ ; https://li...
    MAP_PROJECTION:         EQUIDISTANT CYLINDRICAL
    SOUTH_WEST_CORNER_LAT:  7.005000114440918
    SOUTH_WEST_CORNER_LON:  -168.9949951171875
    DX:                     0.009999999776482582
    DY:                     0.009999999776482582

<xarray.Dataset> Size: 3GB
Dimensions:   (time: 1, lat: 6500, lon: 11700)
Coordinates:
  * time      (time) datetime64[ns] 8B 2001-01-04
    lon       (lon) float32 47kB ManifestArray<shape=(11700,), dtype=float32,...
    lat       (lat) float32 26kB ManifestArray<shape=(6500,), dtype=float32, ...
Data variables:
    Tair      (time, lat, lon) float32 304MB ManifestArray<shape=(1, 6500, 11...
    Tair_min  (time, lat, lon) float32 304MB ManifestArray<shape=(1, 6500, 11...
    Tair_max  (time, lat, lon) float32 304MB ManifestArray<shape=(1, 6500, 11...
    Qair      (time, lat, lon) float32 304MB ManifestArray<shape=(1, 6500, 11...
    PSurf     (time, lat, lon) float32 304MB ManifestArray<shape=(1, 6500, 11...
    LWdown    (time, lat, lon) float32 304MB ManifestArray<shape=(1, 6500, 11...
    SWdown    (time, lat, lon) float32 304MB ManifestArray<shape=(1, 6500, 11...
    Wind_N    (time, lat, lon) float32 304MB ManifestArray<shape=(1, 6500, 11...
    Wind_E    (time, lat, lon) float32 304MB ManifestArray<shape=(1, 6500, 11...
    Rainf     (time, lat, lon) float32 304MB ManifestArray<shape=(1, 6500, 11...
Attributes: (12/17)
    missing_value:          -9999.0
    time_definition:        daily
    shortname:              NLDAS_FOR0010_D_3.0
    title:                  NLDAS Forcing Data L4 Daily 0.01 x 0.01 degree V3...
    version:                3.0 beta
    institution:            NASA GSFC
    ...                     ...
    websites:               https://ldas.gsfc.nasa.gov/nldas/v3/ ; https://li...
    MAP_PROJECTION:         EQUIDISTANT CYLINDRICAL
    SOUTH_WEST_CORNER_LAT:  7.005000114440918
    SOUTH_WEST_CORNER_LON:  -168.9949951171875
    DX:                     0.009999999776482582
    DY:                     0.009999999776482582

In [27]:
vds = open_virtual_mfdataset(
    urls,
    parser=parser,
    registry=registry,
    parallel="lithops",
    combine_attrs=combine_attrs
)

2025-08-26 19:23:25,213 [INFO] config.py:139 -- Lithops v3.6.1 - Python3.12
2025-08-26 19:23:25,214 [INFO] localhost.py:39 -- Localhost storage client created
2025-08-26 19:23:25,215 [INFO] localhost.py:78 -- Localhost compute v2 client created
2025-08-26 19:23:25,455 [INFO] config.py:139 -- Lithops v3.6.1 - Python3.12
2025-08-26 19:23:25,456 [INFO] localhost.py:39 -- Localhost storage client created
2025-08-26 19:23:25,456 [INFO] localhost.py:78 -- Localhost compute v2 client created
2025-08-26 19:23:25,634 [INFO] invokers.py:119 -- ExecutorID c2f8b6-6 | JobID M000 - Selected Runtime: /home/jovyan/rasi-icechunk/.venv/bin/python 
2025-08-26 19:23:25,639 [INFO] invokers.py:186 -- ExecutorID c2f8b6-6 | JobID M000 - Starting function invocation: _open() - Total: 3 activations
2025-08-26 19:23:25,642 [INFO] invokers.py:225 -- ExecutorID c2f8b6-6 | JobID M000 - View execution logs at /tmp/lithops-root/logs/c2f8b6-6-M000.log
2025-08-26 19:23:25,643 [INFO] executors.py:494 -- ExecutorID c2f8b

++++++++++++++++++++++++++++++++++++++++++
Combined history from ['created on date: 2025-06-07T15:04:40', 'created on date: 2025-06-07T15:10:57', 'created on date: 2025-06-07T15:17:23'] to created on date: 2025-06-07T15:04:40||created on date: 2025-06-07T15:10:57||created on date: 2025-06-07T15:17:23
++++++++++++++++++++++++++++++++++++++++++
Combined end_date from ['20010101', '20010102', '20010103'] to 20010103
Combined begin_date from ['20010101', '20010102', '20010103'] to 20010101
++++++++++++++++++++++++++++++++++++++++++
Combined vmin from [228.31163024902344, 227.43487548828125, 227.13450622558594] to 227.13450622558594
Combined vmax from [304.30316162109375, 304.467041015625, 303.4953308105469] to 304.467041015625
++++++++++++++++++++++++++++++++++++++++++
Combined vmin from [227.1617889404297, 225.08892822265625, 223.27415466308594] to 223.27415466308594
Combined vmax from [300.4527282714844, 300.4990539550781, 300.9095764160156] to 300.9095764160156
+++++++++++++++++++++++++

/home/jovyan/nldas-icechunk/.venv/lib/python3.12/site-packages/numcodecs/zarr3.py:145: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)


In [28]:
vds

<xarray.Dataset> Size: 9GB
Dimensions:   (time: 3, lat: 6500, lon: 11700)
Coordinates:
  * lon       (lon) float64 94kB -169.0 -169.0 -169.0 ... -52.03 -52.01 -52.0
  * lat       (lat) float64 52kB 7.005 7.015 7.025 7.035 ... 71.97 71.98 71.99
  * time      (time) datetime64[ns] 24B 2001-01-02 2001-01-03 2001-01-04
Data variables:
    Tair      (time, lat, lon) float32 913MB ManifestArray<shape=(3, 6500, 11...
    Tair_min  (time, lat, lon) float32 913MB ManifestArray<shape=(3, 6500, 11...
    Tair_max  (time, lat, lon) float32 913MB ManifestArray<shape=(3, 6500, 11...
    Qair      (time, lat, lon) float32 913MB ManifestArray<shape=(3, 6500, 11...
    PSurf     (time, lat, lon) float32 913MB ManifestArray<shape=(3, 6500, 11...
    LWdown    (time, lat, lon) float32 913MB ManifestArray<shape=(3, 6500, 11...
    SWdown    (time, lat, lon) float32 913MB ManifestArray<shape=(3, 6500, 11...
    Wind_N    (time, lat, lon) float32 913MB ManifestArray<shape=(3, 6500, 11...
    Wind_E    (time, lat, lon) float32 913MB ManifestArray<shape=(3, 6500, 11...
    Rainf     (time, lat, lon) float32 913MB ManifestArray<shape=(3, 6500, 11...
Attributes: (12/17)
    DX:                     0.009999999776482582
    reference:              Maina-et-al_to-be-submitted; doi:TBD
    SOUTH_WEST_CORNER_LAT:  7.005000114440918
    title:                  NLDAS Forcing Data L4 Daily 0.01 x 0.01 degree V3...
    shortname:              NLDAS_FOR0010_D_3.0
    time_definition:        daily
    ...                     ...
    DY:                     0.009999999776482582
    websites:               https://ldas.gsfc.nasa.gov/nldas/v3/ ; https://li...
    institution:            NASA GSFC
    history:                created on date: 2025-06-07T15:04:40||created on ...
    missing_value:          -9999.0
    doi:                    TBD

In [9]:
# reproducer for https://github.com/virtual-zarr/nldas-icechunk/issues/3
import fsspec
import xarray as xr

data_dir = "s3://nasa-waterinsight/NLDAS3/forcing/daily/"

# Use fsspec to list files in the S3 bucket
fs = fsspec.filesystem("s3", anon=True)
files = fs.glob(data_dir + "**/*.nc")

with fsspec.open('s3://'+files[0]) as f:
    ds = xr.open_dataset(f)
ds

<xarray.Dataset> Size: 3GB
Dimensions:   (lon: 11700, lat: 6500, time: 1)
Coordinates:
  * lon       (lon) float32 47kB -169.0 -169.0 -169.0 ... -52.03 -52.01 -52.0
  * lat       (lat) float32 26kB 7.005 7.015 7.025 7.035 ... 71.97 71.98 71.99
  * time      (time) datetime64[ns] 8B 2001-01-02
Data variables:
    Tair      (time, lat, lon) float32 304MB ...
    Tair_min  (time, lat, lon) float32 304MB ...
    Tair_max  (time, lat, lon) float32 304MB ...
    Qair      (time, lat, lon) float32 304MB ...
    PSurf     (time, lat, lon) float32 304MB ...
    LWdown    (time, lat, lon) float32 304MB ...
    SWdown    (time, lat, lon) float32 304MB ...
    Wind_N    (time, lat, lon) float32 304MB ...
    Wind_E    (time, lat, lon) float32 304MB ...
    Rainf     (time, lat, lon) float32 304MB ...
Attributes: (12/17)
    missing_value:          -9999.0
    time_definition:        daily
    shortname:              NLDAS_FOR0010_D_3.0
    title:                  NLDAS Forcing Data L4 Daily 0.01 x 0.01 degree V3...
    version:                3.0 beta
    institution:            NASA GSFC
    ...                     ...
    websites:               https://ldas.gsfc.nasa.gov/nldas/v3/ ; https://li...
    MAP_PROJECTION:         EQUIDISTANT CYLINDRICAL
    SOUTH_WEST_CORNER_LAT:  7.005
    SOUTH_WEST_CORNER_LON:  -168.995
    DX:                     0.01
    DY:                     0.01